In [13]:
%pip install -U langchain langsmith pandas seaborn --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%env LANGCHAIN_API_KEY="ls__c0f826e0c4514d678c5fdec4a48e6b92"

env: LANGCHAIN_API_KEY="ls__c0f826e0c4514d678c5fdec4a48e6b92"


In [15]:
from langsmith import Client
import pandas as pd

client = Client()

def extract_data(project_name, test_num):
    runs = list(
    client.list_runs(
        project_name=project_name,
        is_root=True,
        error=False
        )
    )

    df = pd.DataFrame(
        [
            {
                **run.inputs,
                "latency": (run.end_time - run.start_time).total_seconds()
                if run.end_time
                else None,  # Pending runs have no end time
                "total_tokens": run.total_tokens,
                "cost": run.completion_cost,
                "run_id": run.id  # Add run ID to the DataFrame
            }
            for run in runs
        ],
        index=[run.id for run in runs],
    )
    df = df.head(test_num)

    # Reverse the DataFrame rows
    df = df.iloc[::-1]

    # Calculate the size of each subset
    subset_size = len(df) // 3

    # Split the DataFrame into three equal parts
    df1 = df.iloc[:subset_size][['input', 'latency', 'total_tokens', 'cost', 'run_id']]
    df2 = df.iloc[subset_size:2*subset_size][['input', 'latency', 'total_tokens', 'cost', 'run_id']]
    df3 = df.iloc[2*subset_size:3*subset_size][['input', 'latency', 'total_tokens', 'cost', 'run_id']]

    # Check if there are any leftover rows and handle them as needed
    if len(df) % 3 != 0:
        # Here you could append the remaining rows to one of the datasets
        # For example, appending to the last dataset
        remainder = df.iloc[3*subset_size:]
        df3 = pd.concat([df3, remainder])

    # Save to Excel files
    df1.to_excel(f'Results\comparison_results\Easy_comparison\{project_name}_Easy_langsmith_extraction.xlsx', index=False, engine='openpyxl')
    df2.to_excel(f'Results\comparison_results\Medium_comparison\{project_name}_Medium_langsmith_extraction.xlsx', index=False, engine='openpyxl')
    df3.to_excel(f'Results\comparison_results\Hard_comparison\{project_name}_Hard_langsmith_extraction.xlsx', index=False, engine='openpyxl')
    


In [16]:
# extract_data("Normal", 30)
extract_data("ReAct", 30)
